In [2]:
import pandas as pd
import scipy.stats as stats

# Load the uploaded CSV file
file_path = "../result/accuracy.csv"
df = pd.read_csv(file_path)

# List of metric columns
metrics = ['Train Strict Accuracy (%)', 'Train ±1 Grade Accuracy (%)', 'Val Strict Accuracy (%)', 'Val ±1 Grade Accuracy (%)']

# Function to compute statistics
def compute_stats(group):
    stats_dict = {}
    n = len(group)
    for col in metrics:
        mean = group[col].mean()
        std = group[col].std(ddof=1)
        se = std / (n ** 0.5)
        margin_of_error = stats.t.ppf(0.975, df=n-1) * se  # 95% confidence
        stats_dict[f'{col}_mean'] = mean
        stats_dict[f'{col}_moe'] = margin_of_error
    return pd.Series(stats_dict)

# Group by model and compute statistics
summary_df = df.groupby('model').apply(compute_stats).reset_index()

# Save the result to a new CSV file
summary_df.to_csv("analyze_result.csv", index=False)

# Show result
summary_df


/var/folders/jy/g825nyfd36jghgh8wg11sll00000gn/T/ipykernel_92817/959816039.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary_df = df.groupby('model').apply(compute_stats).reset_index()


,model,Train Strict Accuracy (%)_mean,Train Strict Accuracy (%)_moe,Train ±1 Grade Accuracy (%)_mean,Train ±1 Grade Accuracy (%)_moe,Val Strict Accuracy (%)_mean,Val Strict Accuracy (%)_moe,Val ±1 Grade Accuracy (%)_mean,Val ±1 Grade Accuracy (%)_moe
0,adaboost_deepset,46.6432,0.216833,83.5040,0.268362,45.2184,0.344054,82.0140,0.299346
1,adaboost_deepset_xy,46.5388,0.235521,83.1020,0.234141,45.0884,0.406285,81.5636,0.329595
2,adaboost_deepset_xy_additive,45.1780,0.244329,82.1824,0.242334,43.7084,0.302413,80.1520,0.273986
3,adaboost_set_transformer,52.5720,0.207748,84.4008,0.172015,46.6244,0.361495,81.3252,0.295044
4,adaboost_set_transformer_additive,48.7264,0.358538,82.9448,0.230831,45.3636,0.358239,81.2336,0.270255
5,adaboost_set_transformer_xy,52.2924,0.205671,84.3016,0.170054,46.2928,0.355469,81.5876,0.232923
6,deepset,45.4856,0.431579,82.6028,0.656453,43.1096,0.366516,80.3992,0.707042
7,deepset_xy,45.0200,0.487367,82.3812,0.562741,42.5632,0.399284,80.0868,0.564597
8,deepset_xy_additive,45.5496,0.492921,82.4384,0.714152,43.2988,0.562717,80.4772,0.778514
9,gbm_ensemble_all,80.8524,0.242009,95.6236,0.111793,48.8432,0.346286,83.2768,0.236187


In [2]:
import pandas as pd
import numpy as np

def process_ordinal_results(input_path, output_path_combined):
    # 1. Read Excel
    df = pd.read_excel(input_path)

    # 2. Compute mean & std by (model, threshold)
    summary = (
        df.groupby(["model", "threshold"])["accuracy"]
          .agg(["mean", "std"])
          .reset_index()
    )

    # 3. Create a "mean ± std" string
    summary["combined"] = summary.apply(
        lambda row: f"{row['mean']:.2f} ± {row['std']:.2f}",
        axis=1
    )

    # 4. Convert to wide format (like your screenshot)
    combined_wide = summary.pivot(
        index="model",
        columns="threshold",
        values="combined"
    )

    # 5. Sort columns in natural V4 → V13 order
    def sort_key(col):
        try:
            return int(col.split("V")[1].replace(")", ""))
        except:
            return 999

    combined_wide = combined_wide.reindex(
        sorted(combined_wide.columns, key=sort_key),
        axis=1
    )

    # 6. Save to CSV
    combined_wide.to_csv(output_path_combined)

    return combined_wide



# Usage
result_df = process_ordinal_results(
    "../result/ordinal_result.xlsx",
    "ordinal_result_summary.csv"
)
print(result_df)


threshold                                  P(>V4)        P(>V5)        P(>V6)  \
model                                                                           
deepset_ordinal                      81.51 ± 0.39  80.77 ± 0.32  83.81 ± 0.23   
deepset_ordinal_xy                   81.51 ± 0.42  80.71 ± 0.23  83.80 ± 0.27   
deepset_ordinal_xy_additive          81.31 ± 0.37  80.71 ± 0.44  83.86 ± 0.28   
ordinal_adaboost_ensemble            84.06 ± 0.31  82.38 ± 0.34  85.81 ± 0.31   
ordinal_gbm_ensemble                 83.91 ± 0.57  82.61 ± 0.39  85.87 ± 0.69   
ordinal_geometric_mean_ensemble      83.93 ± 0.36  82.55 ± 0.29  85.73 ± 0.38   
ordinal_lightgbm_ensemble            83.52 ± 0.54  82.35 ± 0.46  85.68 ± 0.69   
ordinal_median_ensemble              83.01 ± 0.33  81.77 ± 0.30  84.83 ± 0.31   
ordinal_soft_voting_ensemble         84.06 ± 0.31  82.38 ± 0.34  85.81 ± 0.31   
ordinal_stacking_ensemble            84.20 ± 0.50  82.51 ± 0.56  85.84 ± 0.62   
ordinal_trimmed_mean_ensembl